# Freespace Garment Simulation

Notebook for prepping the freespace simulation. This does NOT do rendering. Only simulation

In [1]:
import os
import sys
from pathlib import Path
import pickle

import bpy
import mathutils
import numpy as np

# Imports from this repository
sys.path.append("../../")
sys.path.append("../")
from simulation.cloth_3d_util.accessor import Cloth3DCanonicalAccessor
from simulation.cloth_3d_util.util import loadInfo
from simulation.pipeline.simulate_garment_hanging_rest_state import \
    simulate_garment_hanging_rest_state
from simulation.blender_util_dylan.physics import set_sim_output_as_default_mesh_shape
from simulation.blender_util_dylan.checkpointer import BlendFileCheckpointer
from simulation.blender_util_dylan.gripper import GripperAnimation
from simulation.blender_util_dylan.debug import print_obj_keyframe_coordinates

%load_ext autoreload
%autoreload 2

WARN (bgl): source/blender/python/generic/bgl.c:2654 BPyInit_bgl: 'bgl' imported without an OpenGL backend. Please update your add-ons to use the 'gpu' module. In Blender 4.0 'bgl' will be removed.


/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  retur

In [2]:
FILE_ROOT = Path(os.getcwd())
CLOTH3D_PATH = Path(os.path.expanduser("~/DataLocker/datasets/CLOTH3D/training/"))
OUTPUT_ROOT = FILE_ROOT / ".." / "script_output" / "full_dataset_attempt_2"
GARMENTNETS_SAMPLE_DATASET_ZARR = (FILE_ROOT / ".." / ".." / "data" / 
                                   "garmentnets_simulation_dataset_sample.zarr")
assert (GARMENTNETS_SAMPLE_DATASET_ZARR.exists())
GARMENTNETS_SAMPLE_PATH = ["Tshirt", "samples"]

# Make the output directory if it doesn't exist.
OUTPUT_ROOT.mkdir(exist_ok=True)


PLANE_OFFSET = 0.025  # [m]

## Find All Configurations

In [3]:
# List the directory the garmentnets Tshirt samples are in. The naming convention is 
# <sample_id>_Tshirt_<gripped_index>
sample_configs = []
for sample_dir in (GARMENTNETS_SAMPLE_DATASET_ZARR / "Tshirt" / "samples").iterdir():
    if sample_dir.name.startswith('.'):
        # Skip the .zgroup that makes it a Zarr dataset
        continue
    print(sample_dir.name)
    sample_id, garment_name, grip_vertex_idx = sample_dir.name.split('_')
    if garment_name == "Tshirt":
        sample_configs.append({
            "sample_id": sample_id,
            "garment_name": garment_name,
            "grip_vertex_idx": int(grip_vertex_idx)
        })
    else:
        print("Found non-Tshirt garment in directory. Skipping")



00471_Tshirt_153
07135_Tshirt_4909
07516_Tshirt_2950
04199_Tshirt_3855
04956_Tshirt_2092
06698_Tshirt_321
01841_Tshirt_2878
05440_Tshirt_1903
03685_Tshirt_3008
06305_Tshirt_44
05739_Tshirt_174
04571_Tshirt_4765
04325_Tshirt_3309
05839_Tshirt_3309
00387_Tshirt_2659
03965_Tshirt_3646
07516_Tshirt_3262
04891_Tshirt_4354
05542_Tshirt_3908
06105_Tshirt_80
05739_Tshirt_3482
03966_Tshirt_34
06165_Tshirt_2709
02952_Tshirt_3909
00380_Tshirt_509


In [4]:
print("Sample IDs:", [i["sample_id"] for i in sample_configs])
print("Garment Types:", [i["garment_name"] for i in sample_configs])
print("Gripped vertices:", [i["grip_vertex_idx"] for i in sample_configs])

Sample IDs: ['00471', '07135', '07516', '04199', '04956', '06698', '01841', '05440', '03685', '06305', '05739', '04571', '04325', '05839', '00387', '03965', '07516', '04891', '05542', '06105', '05739', '03966', '06165', '02952', '00380']
Garment Types: ['Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt', 'Tshirt']
Gripped vertices: [153, 4909, 2950, 3855, 2092, 321, 2878, 1903, 3008, 44, 174, 4765, 3309, 3309, 2659, 3646, 3262, 4354, 3908, 80, 3482, 34, 2709, 3909, 509]


## Loop Through The Configurations And Simulate The Resting State

In [6]:
def simulate_resting_state_if_not_exists(config: dict, sample_data: dict, 
                                         checkpointer: BlendFileCheckpointer, 
                                         frames_to_resting_state: int=120):
    # Confirmed to work as expected!
    # Reset to empty blender environment. 
    bpy.ops.wm.read_factory_settings()

    if checkpointer.does_rest_state_checkpoint_exist():
        print("Hanging state checkpoint exists. Skipping.")
        # Load the checkpoint instead of simulating the resting state again.
        result_data_smpl = checkpointer.load_hanging_rest_state()
    else:
        print("Hanging state checkpoint does not exist. Simulating and saving checkpoint.")
        # Run the SMPL simulation to get the garment in a stationary hanging configuration
        result_data_smpl = simulate_garment_hanging_rest_state(config, sample_data, frames_to_resting_state)

        # Then set the default mesh shape to this shape
        cloth_obj = bpy.data.objects["cloth"]
        set_sim_output_as_default_mesh_shape(cloth_obj, frames_to_resting_state)

        # Now create a checkpoint for this so we don't have to resimulate every time we run this
        # notebook.
        checkpointer.save_hanging_rest_state(overwrite_ok=True, result_data_smpl=result_data_smpl)



In [7]:
for config in sample_configs:
    ## Setup the simulation.
    accessor = Cloth3DCanonicalAccessor(CLOTH3D_PATH)
    sample_key = f"{config['sample_id']}_{config['garment_name']}_{config['grip_vertex_idx']}"
    sample_dir = OUTPUT_ROOT / sample_key
    sample_dir.mkdir(exist_ok=True)

    # result_file = "simulation_result.pk"
    # result_path = sample_dir / result_file

    # Get a dictionary containing the data for this sample garment.
    sample_data = accessor.get_sample_data(**config)

    garment_info_mat_filename = CLOTH3D_PATH / config["sample_id"] / "info.mat"
    garment_info = loadInfo(garment_info_mat_filename)

    checkpointer = BlendFileCheckpointer(sample_dir, save_new_checkpoints=False)

    simulate_resting_state_if_not_exists(config, sample_data, checkpointer)

Hanging state checkpoint does not exist. Simulating and saving checkpoint.
Running SMPL Simulation Pipeline
--------------------------------------------------------------------------------
bake: frame 0 :: 120
bake: frame 1 :: 120
bake: frame 2 :: 120
bake: frame 3 :: 120
bake: frame 4 :: 120
bake: frame 5 :: 120
bake: frame 6 :: 120
bake: frame 7 :: 120
bake: frame 8 :: 120
bake: frame 9 :: 120
bake: frame 10 :: 120
bake: frame 11 :: 120
bake: frame 12 :: 120
bake: frame 13 :: 120
bake: frame 14 :: 120
bake: frame 15 :: 120
bake: frame 16 :: 120
bake: frame 17 :: 120
bake: frame 18 :: 120
bake: frame 19 :: 120
bake: frame 20 :: 120
bake: frame 21 :: 120
bake: frame 22 :: 120
bake: frame 23 :: 120
bake: frame 24 :: 120
bake: frame 25 :: 120
bake: frame 26 :: 120
bake: frame 27 :: 120
bake: frame 28 :: 120
bake: frame 29 :: 120
bake: frame 30 :: 120
bake: frame 31 :: 120
bake: frame 32 :: 120
bake: frame 33 :: 120
bake: frame 34 :: 120
bake: frame 35 :: 120
bake: frame 36 :: 120
bake: f

## Generate The Dynamics Commands 

This is a dictionary where the keys are the full sample keys `<sample_id>_Tshirt_<gripped_idx>`

In [8]:
NUM_ACTION_SEQS_PER_SAMPLE = 5

VELOCITY_MAX = 0.2
VELOCITY_MIN = 0.05

MOTIONS_PER_SEQUENCE = 3

DURATION_MIN = 20
DURATION_MAX = 70

# velocity_mps = 0.15 if using_simple_dynamics else 0.75

# direction_vecs = [
#     np.array((1.0, 0.0, 0.0)),
#     np.array((0.0, -1.0, 0.0)),
#     np.array((0.0, 0.0, -1.0)),
#     np.array((-1.0, -1.0, 1.0))
# ]

# frame_duration = 5

dynamics_control_sequences = dict()

for config in sample_configs:
    sample_key = f"{config['sample_id']}_{config['garment_name']}_{config['grip_vertex_idx']}"
    
    dynamics_control_sequences[sample_key] = []

    for seq_num in range(NUM_ACTION_SEQS_PER_SAMPLE):
        control_seq = []
        for i in range(MOTIONS_PER_SEQUENCE):
            vel = VELOCITY_MIN + (VELOCITY_MAX - VELOCITY_MIN) * np.random.rand()

            direction = np.random.rand(3)
            direction /= np.linalg.norm(direction)

            duration = np.random.randint(DURATION_MIN, DURATION_MAX)

            control_seq.append({
                "direction_vec": direction,
                "velocity": vel,
                "frame_duration": duration
            })
        dynamics_control_sequences[sample_key].append(control_seq)
# print(dynamics_control_sequences)


        



In [9]:
for sample_key, control_seqs in dynamics_control_sequences.items():
    # print(sample_key)
    sample_dir = OUTPUT_ROOT / sample_key
    
    for i, seq in enumerate(control_seqs):
        control_seq_dir = sample_dir / f"dynamics_seq_{i}"
        control_seq_dir.mkdir(exist_ok=False)

        # Write the control sequence to the directory so we can load it later.
        control_seq_file = control_seq_dir / "control_sequence.pkl"
        pickle.dump(seq, control_seq_file.open('wb'))

## Run The Dynamics Routine

**Don't actually run this in a notebook!** Prints too much and crashes.

In [ ]:
# for config in sample_configs:
#     ## Setup the simulation.
#     sample_key = f"{config['sample_id']}_{config['garment_name']}_{config['grip_vertex_idx']}"
#     # control_sequences = dynamics_control_sequences[sample_key]
#     print("Simulating Dynamics for Sample:", sample_key)

    
#     accessor = Cloth3DCanonicalAccessor(CLOTH3D_PATH)
#     sample_dir = OUTPUT_ROOT / sample_key
#     sample_dir.mkdir(exist_ok=True)

#     # Get a dictionary containing the data for this sample garment.
#     sample_data = accessor.get_sample_data(**config)

#     garment_info_mat_filename = CLOTH3D_PATH / config["sample_id"] / "info.mat"
#     garment_info = loadInfo(garment_info_mat_filename)

#     checkpointer = BlendFileCheckpointer(sample_dir, save_new_checkpoints=False)

#     for seq_idx in range(NUM_ACTION_SEQS_PER_SAMPLE):
#         print("Simulating Control Sequence", seq_idx)

#         # Reset the environment to the simulated resting state
#         checkpointer.load_hanging_rest_state()
        
#         seq_dir = sample_dir / f"dynamics_seq_{seq_idx}"
#         assert (seq_dir.exists())

#         control_sequence_path = seq_dir / "control_sequence.pkl"
#         control_sequence = pickle.load(control_sequence_path.open('rb'))

#         # Create the gripper animation controller
#         cloth_obj = bpy.data.objects["cloth"]
#         gripper_obj = bpy.data.objects["Empty"]
#         gripper_animation = GripperAnimation(gripper_obj, cloth_obj)

#         # for vec in direction_vecs:
#         for action in control_sequence:
#             gripper_animation.add_movement(action["direction_vec"], action["velocity"], 
#                                            action["frame_duration"])
        
#         # Now bake the sim.
#         bpy.ops.ptcache.bake_all()

#         # Get the ground truth mesh for each frame in the animation.
#         # Actually, this isn't necessary if we just save the final blend file state.
#         # bpy.context.scene.frame_set()

#         # Finally, save the final simulation blend file state so we can easily render later on.
#         final_blend_path = seq_dir / "dynamics_animation.blend"
#         bpy.ops.wm.save_as_mainfile(filepath=final_blend_path.as_posix())

        
    